#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 2:  Link Analysis -- HITS + SEO

### 100 points [5% of your final grade]

### Due: Sunday, February 25, 2018 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test HITS algortihm over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Wednesday, February 28, 2018 at 11:59pm.

# Part 1: HITS (70 points)

## A re-Tweet Graph

In this assignment, we're going to adapt the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *HITS.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are weighted and directed. If Bob retweets Alice's tweets 10 times, there is an edge from Bob to Alice with weight 10, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the HITS algorithm on the graph you build here.


In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt  


ds_tweets = pd.read_json("HITS.json", lines=True)
ds_tweets_50 = ds_tweets
ds_tweets_50

,created_at,entities,id,retweeted_status,source,text,user
0,2015-03-04 07:55:13,"{u'user_mentions': [{u'id': 3039321886, u'scre...",573028910927089664,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @tgbtgbtgbtgb999: يا انصار #الدولة_الإسلامي...,"{u'favourites_count': 4, u'description': u'{ال..."
1,2015-03-04 20:57:46,"{u'user_mentions': [{u'id': 3065514742, u'scre...",573225847877967873,"{u'source': u'<a href=""http://twitter.com/#!/d...","<a href=""http://twitter.com/download/android"" ...",RT @jolaybibdawla9: #هام #للنشر\n: دخول دبابا...,"{u'favourites_count': 79, u'description': u'يا..."
2,2015-03-10 21:48:19,"{u'user_mentions': [{u'id': 3068706044, u'scre...",575412894180982785,"{u'created_at': u'2015-03-10T21:44:35.000Z', u...","<a href=""http://twitter.com/download/android"" ...",RT @anbaar4: @Altawheed21333 اللهم رب السموات ...,"{u'favourites_count': 30, u'name': u'MEZHOUD G..."
3,2015-03-11 09:52:14,"{u'user_mentions': [{u'id': 3058557188, u'scre...",575595075381555200,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com/download/iphone"" r...",RT @qese2: بشرى قادمة بإذن الله ستكون قاصمة لظ...,"{u'favourites_count': 31, u'name': u'سآم', u'f..."
4,2015-03-13 18:32:26,"{u'user_mentions': [{u'id': 2683520216, u'scre...",576450764278640640,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com/download/android"" ...",RT @o_m_a_t_y_5: انيس الموحدين\nبعد هذه الأيام...,"{u'favourites_count': 360, u'description': u'‏..."
5,2015-03-15 18:38:42,"{u'user_mentions': [{u'id': 3039321886, u'scre...",577177115533164544,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com/#!/download/ipad"" ...",RT @uhbuhbuhb555: اقدام تستحق التقدير\nاقدام ت...,"{u'favourites_count': 13, u'name': u'أحمد العر..."
6,2015-03-15 22:19:07,"{u'user_mentions': [{u'id': 2683520216, u'scre...",577232584507293696,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com/download/iphone"" r...",RT @omaty1_2: #دولة_الخلافة\n#ولاية_برقة\n\n[ا...,"{u'favourites_count': 235, u'name': u'المجاهد١..."
7,2015-03-16 01:11:09,"{u'user_mentions': [{u'id': 3058557188, u'scre...",577275877366607872,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com/download/android"" ...",RT @QeSe5: عِندما بُشرّ بقرب عمليته طارّ فرحاً...,"{u'favourites_count': 2720, u'description': u'..."
8,2015-03-16 16:19:06,"{u'user_mentions': [{u'id': 2878363620, u'scre...",577504374085169152,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com/download/android"" ...",RT @AldolimyOdii: وزير الداخلية العراقي : أوقف...,"{u'favourites_count': 7, u'name': u'موحدون ', ..."
9,2015-03-16 18:12:47,"{u'user_mentions': [{u'id': 3050302429, u'scre...",577532982967312386,"{u'source': u'<a href=""http://twitter.com/down...","<a href=""http://twitter.com/download/android"" ...",RT @joker_meslim_12: اخواتي.\nلا تدخلن في جدال...,"{u'favourites_count': 24, u'description': u'‏‏..."


In [2]:
user_ids = map(lambda d: d['id'], ds_tweets_50.user)
user_ids = np.array(user_ids)

source_ids = map(lambda d: d['user']['id'], ds_tweets_50.retweeted_status)
source_ids = np.array(source_ids)

user_ids_unique = list(set(list(set(user_ids)) + list(set(source_ids))))

In [3]:
N = len(user_ids_unique)
Hits_matrix = np.matrix(np.zeros([N, N]))

for i in range(len(user_ids_unique)):
    for j in range(len(user_ids_unique)):
        if(i == j):
            continue
        else:
            user_i = user_ids_unique[i]
            user_j = user_ids_unique[j] 
            user_i_tweets = user_ids == user_i
            user_i_retweets = source_ids[user_i_tweets]
            user_j_mentions = user_i_retweets[user_i_retweets == user_j]
            l = len(user_j_mentions)
            Hits_matrix[i,j] = l

We will not check the correctness of your graph. However, this will affect the HITS results later.

## HITS Implementation

Your program will return the top 10 users with highest hub and authority scores. The **output** should be like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal scores.
* It is up to you to decide when to terminate the HITS calculation.
* There are HITS implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and HITS calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

In [6]:
A = Hits_matrix
A_t = np.transpose(A)

u = np.ones((len(A), 1))

def normalize(x):
    y = np.squeeze(np.asarray(x))
    y2 = y**2
    y2 = math.sqrt(sum(y2))
    res = y/y2
    res = np.matrix(res.reshape(len(res), 1))
    return res

for i in range(100):
    v = A_t.dot(u)
    u = A.dot(v)
    u = normalize(u)
    v = normalize(v)
    print "v:",np.squeeze(np.asarray(v))
    print "u:",np.squeeze(np.asarray(u))

v: [0.00182744 0.00365487 0.14254004 ... 0.         0.         0.        ]
u: [3.13731995e-03 3.46360123e-04 9.79847768e-03 ... 1.22129591e-02
 3.01182716e-05 1.11437605e-03]
v: [0.00244131 0.00385902 0.08074749 ... 0.         0.         0.        ]
u: [2.58528923e-03 1.41816210e-04 6.03463705e-03 ... 1.01115873e-02
 9.30534300e-07 5.36206633e-04]
v: [0.00200405 0.00355283 0.07533055 ... 0.         0.         0.        ]
u: [2.16365714e-03 1.17745412e-04 5.94806710e-03 ... 8.51761962e-03
 6.51711645e-07 5.12255709e-04]
v: [0.00165622 0.00329872 0.07659091 ... 0.         0.         0.        ]
u: [1.83093908e-03 1.03357674e-04 6.10315428e-03 ... 7.27847846e-03
 5.95234767e-07 5.23050887e-04]
v: [0.00138303 0.00309265 0.07805376 ... 0.         0.         0.        ]
u: [1.57664221e-03 9.29971786e-05 6.22731594e-03 ... 6.33280182e-03
 5.82328675e-07 5.33436553e-04]
v: [0.00117378 0.00292859 0.07912819 ... 0.         0.         0.        ]
u: [1.38593335e-03 8.53130447e-05 6.31172898e-03 .

In [8]:
v_vec = np.squeeze(np.asarray(v))
u_vec = np.squeeze(np.asarray(u))

def sort_by_vec(vec):
    vec_id = zip(vec, user_ids_unique)
    vec_id.sort(reverse=True)
    return vec_id

In [18]:
print "Top 10 Hubs\n\n"
print "score,", "id"
sort_by_vec(u_vec)[:10]

Top 10 Hubs


score, id


[(0.62311860812251, 3068706044),
 (0.29616066155009085, 3093940760),
 (0.25987256424974764, 2194518394),
 (0.20258565674888135, 2862783698),
 (0.17051844912831937, 3092183276),
 (0.16698984616196638, 3029724797),
 (0.14773329728408674, 2990704188),
 (0.1448265194376799, 3001500121),
 (0.12915071372153997, 3086921438),
 (0.1252633748791499, 3042686360)]

In [20]:
print "Top 10 Authorities\n\n"
print "score,", "id"
sort_by_vec(v_vec)[:10]

Top 10 Authorities


score, id


[(0.5446248048789044, 3042570996),
 (0.493216599868788, 3065514742),
 (0.4439307063060392, 1638625987),
 (0.28659130814272554, 3077733683),
 (0.22433355505187025, 3039321886),
 (0.1218841572614654, 3077695572),
 (0.11322781827450255, 3019659587),
 (0.09803152133129726, 1358345766),
 (0.09398530780220558, 3061155846),
 (0.09367926178547761, 3092580049)]

# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **kbeznak parmatonic** --- two terms, lower case, no quote. As of today (Feb 16, 2018), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization#White_hat_versus_black_hat_techniques).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?


What's your strategy? (2-4 paragraphs)


URL: people.tamu.edu/~ilan50_guru/kbeznak+parmatonic/

### STRATEGIES

I googled for SEO best practices and incorporated many of the suggestions from a variety of web sites

#### 1. Use the search term in the URL: 
My url had "kbeznak parmatonic" in it's url itself and that is bound to give it a boost.

#### 2. Keep the URL simple:
I have kept my url as simple as I can as complex urls are not good for rankings

#### 3. Use the term often in your webpage: 
I have used the term pretty often in my web page to increase the relevance of it when queried for kbeznak parmatonic. 

#### 4. Don't overuse it and make it look spammy:
I ensured that my page didn't contain only those 2 words again and again.

#### 5. Use the search term in headings and make it bold:
I ensured that many of my "kbeznak parmatonic"s were in Bold and thus increasing their significance.

#### 6. Use trustworthy outlinks:
I had random links to famous websites like Fox news, Tamu, CS670 and Wikipedia links to Donald Trump. The outlinks also seemed to affect the ranking. 

#### 7. Youtube Link to a "kbeznak parmatonic" video:
I figured that a link to a kbeznak parmatonic video on youtube would make it look even more credible.

#### 8. Keeping the text as real as possible:
I basically copy pasted some random content and replaced a commonly occuring word in that text with kbeznak parmatonic. This would keep it as real as possible to an actual webpage. 

#### 9. Using buzzwords:
I also changed a few of the other commonly words randomly to donald trump and oscars 2018. Using keywords like this would increase it's page rank as it would show up if people googed for those terms. 

#### 10.  Inlink from the youtube video:
A link from the youtube video I created to the page might increase it's rank and make it seem more relevant.  